In [ ]:
import numpy as np
import pandas as pd
from math import lcm
from collections import OrderedDict, defaultdict
import os

In [ ]:
def generate_valid_solution(course_df, T, R, fill_value=0, dtype=np.int16):
    """
    Generate a valid solution using your 1D approach with transpose.
    Parameters:
    - course_df (pd.DataFrame): with 'course_id' and 'val' (duration)
    - T (int): rows
    - R (int): columns
    - fill_value (int): fill for empty slots

    Returns:
    - np.ndarray shape (T, R)
    """
    durations = course_df.set_index('course_id')['duration'].to_dict()
    course_ids = course_df['course_id'].tolist()

    total_slots = T * R
    required_slots = sum(durations.values())
    if required_slots > total_slots:
        raise ValueError("Not enough slots for all courses")

    arr_1d = np.full(total_slots, fill_value, dtype=dtype)

    # Find indices where 2-hour course cannot start:
    invalid_starts = {i for i in range(T * R) if (i % T) % (T//5) == T//5 - 1}

    # Compute valid starting indices for 2-hour courses
    valid_starts = [i for i in range(total_slots) if i not in invalid_starts]

    # Place 2-hour courses first
    used_indices = set()
    for cid in course_ids:
        dur = durations[cid]
        if dur == 2:
            placed = False
            np.random.shuffle(valid_starts)
            for start_idx in valid_starts:
                # Ensure both start and next slot are unused
                if start_idx in used_indices or (start_idx + 1) in used_indices:
                    continue
                # Place the course at start_idx and start_idx + 1
                arr_1d[start_idx] = cid
                arr_1d[start_idx + 1] = cid
                used_indices.update({start_idx, start_idx + 1})
                placed = True
                break
            if not placed:
                raise RuntimeError(f"Cannot place 2-hour course {cid}")

    # Place 1-hour courses
    one_hour_courses = [cid for cid in course_ids if durations[cid] == 1]
    free_positions = [i for i in range(total_slots) if i not in used_indices]
    np.random.shuffle(one_hour_courses)

    np.random.shuffle(one_hour_courses)
    np.random.shuffle(free_positions)

    for cid, pos in zip(one_hour_courses, free_positions):
        arr_1d[pos] = cid
        used_indices.add(pos)

    # The rest remain fill_value

    # Reshape to (R, T), then transpose to (T, R)
    arr_2d = arr_1d.reshape(R, T).T

    return arr_2d


In [ ]:
def array_to_multi_position_dict(arr, ignore_value=0):
    """
    Create a dictionary mapping each non-ignored value to a list of (row, col) positions
    where it appears in the 2D array.

    Parameters:
    - arr (np.ndarray): Input 2D array
    - ignore_value (int): Value to ignore (default=0)

    Returns:
    - dict: {value: [(row1, col1), (row2, col2), ...]}
    """
    from collections import defaultdict

    value_positions = defaultdict(list)

    for row in range(arr.shape[0]):
        for col in range(arr.shape[1]):
            val = arr[row, col]
            if val != ignore_value:
                value_positions[int(val)].append((row+1, col+1))

    return dict(sorted(value_positions.items()))


In [ ]:
def export_to_txt(arr, folder="solutions", filename="solution.txt"):
    # Ensure the folder exists
    os.makedirs(folder, exist_ok=True)

    # Full file path
    filepath = os.path.join(folder, filename)

    # Convert array to string without clipping or wrapping
    with np.printoptions(threshold=np.inf, linewidth=10000):
        arr_str = np.array2string(arr, separator=', ')

    # Write to file
    with open(filepath, 'w') as f:
        f.write(arr_str)

In [ ]:
def import_from_txt(folder="output", filename="output.txt"):
    filepath = os.path.join(folder, filename)

    with open(filepath, 'r') as f:
        content = f.read()

    # Convert string back to numpy array
    arr = np.array(eval(content), dtype=np.int16)  # Use eval because it's safe with known input like this
    return arr

In [ ]:
def get_course_duration(df, course_id):
    row = df[df['course_id'] == course_id]
    if row.empty:
        raise ValueError(f"Course ID '{course_id}' not found in DataFrame.")
    return int(row.iloc[0]['duration'])


In [ ]:
def is_solution_valid(arr, criteria_functions, course_df, ignore_value=0):
    """
    Validate a solution array against multiple criteria.

    Parameters:
    - arr (np.ndarray): 2D solution array.
    - criteria_functions (list of callables): Functions accepting (solution_dict, course_df) and returning bool.
    - course_df (pd.DataFrame): DataFrame with course info.
    - ignore_value (int): Value in arr to ignore (default=0).

    Returns:
    - bool: True if all criteria pass, False otherwise.
    """
    solution_dict = array_to_multi_position_dict(arr, ignore_value)

    for check in criteria_functions:
        if not check(solution_dict, course_df):
            return False
    return True


In [ ]:
def time_slot_criteria(solution_dict, course_df):
    """
    Validate that each course in solution_dict appears the correct number of times
    according to course_df, and that 2-hour courses occupy two consecutive slots
    in the same row.

    Parameters:
    - solution_dict (dict): Mapping from course_id to list of (row, col) positions.
    - course_df (pd.DataFrame): DataFrame with 'course_id' and 'val' columns.

    Returns:
    - bool: True if all courses satisfy their duration and consecutive slot criteria.
    """
    for course_id, positions in solution_dict.items():
        expected_count = get_course_duration(course_df, course_id)
        
        # Check course duration count
        if len(positions) != expected_count:
            print(f"Failed on first test: {course_id}")
            return False
        
        # Additional check for 2-hour courses
        if expected_count == 2:
            (r1, c1), (r2, c2) = positions
            if c1 != c2 or abs(r1 - r2) != 1:
                print(f"Failed on second test: {course_id}")
                return False

    return True


In [ ]:
def dict_to_course_df(schedule_dict, course_code):
    rows = []
    for course_id, slots in schedule_dict.items():
        duration = len(slots)
        time_slots = sorted(ts for ts, _ in slots)
        room_ids = set(rid for _, rid in slots)

        start_time_slot = time_slots[0]
        room_id = room_ids.pop() if len(room_ids) == 1 else list(room_ids)

        rows.append({
            'course_id': course_id,
            'course_code': course_code[course_id-1],
            'duration': duration,
            'time_slot_id': start_time_slot,
            'room_id': room_id
        })
    return pd.DataFrame(rows)


In [ ]:
def get_used_rooms(solution_dict):
    used_rooms = set()
    for slots in solution_dict.values():
        if slots:
            _, room_id = slots[0]
            used_rooms.add(room_id)
    return sorted(used_rooms)

In [ ]:
def get_combinations(mods):
    """
    Get all unique class assignment combinations (1-based), sorted.

    Parameters:
    - mods (list of int): number of classes for each course.

    Returns:
    - list of tuples: sorted list of unique combinations (starting from 1)
    """
    cycle_length = lcm(*mods)
    seen = set()
    for student_id in range(cycle_length):
        combo = tuple((student_id % n) + 1 for n in mods)
        seen.add(combo)
    return sorted(seen)


In [ ]:
def filter_classes_by_combination(df, course_order, combination):
    """
    Given a class combination, return the subset of df matching that class assignment.

    Parameters:
    - df (pd.DataFrame): DataFrame with 'course_code' column
    - course_order (list of str): ordered course names corresponding to combination
    - combination (tuple of int): class assignment (1-based) per course

    Returns:
    - pd.DataFrame: filtered rows from df
    """
    result_frames = []
    
    for course_name, class_number in zip(course_order, combination):
        # Format class_number to two digits, e.g., 1 → "01"
        class_suffix = f"{class_number:02}"
        
        # Match course_code like "MA1201-16001X" where X = class number
        matching_rows = df[df['course_code'].str.startswith(f"{course_name}-") &
                           df['course_code'].str.contains(f"{course_name}-160{class_suffix}")]
        result_frames.append(matching_rows)
    
    # Concatenate all results into one DataFrame
    return pd.concat(result_frames, ignore_index=True)


In [ ]:
def add_time_range_column(df):
    """
    Add a 'time_range' column to the DataFrame using 'time_slot' and 'duration' columns.

    Parameters:
    - df (pd.DataFrame): Must include 'time_slot' and 'duration' columns.

    Returns:
    - pd.DataFrame: Original DataFrame with an added 'time_range' column.
    """
    def slot_to_time_range(time_slot, duration):
        start_hour = int(time_slot[1:])
        end_hour = start_hour + duration
        return f"{start_hour:02}.00-{end_hour:02}.00"

    df = df.copy()
    df['time_range'] = df.apply(lambda row: slot_to_time_range(row['time_slot'], row['duration']), axis=1)
    return df


In [ ]:
def group_schedule_by_day(df):
    """
    Group the schedule DataFrame by day (ordered: Senin to Jumat).

    Returns:
        OrderedDict: {
            'Senin': [(time_range, course_code_prefix, room_name), ...],
            ...
        }
    """
    day_order = ['Senin', 'Selasa', 'Rabu', 'Kamis', 'Jumat']
    temp = defaultdict(list)

    for _, row in df.iterrows():
        entry = (row['time_range'], row['course_code'][:6], row['room_name'])
        temp[row['day']].append(entry)

    # Sort each day's entries by start time
    def time_key(item):
        return int(item[0][:2])  # Extract hour from "HH.00-HH.00"

    result = OrderedDict()
    for day in day_order:
        if day in temp:
            result[day] = sorted(temp[day], key=time_key)

    return result


In [ ]:
fmipa = pd.read_csv('csv/fmipa.csv')
time_slots = pd.read_csv('csv/time_slots.csv')
rooms = pd.read_csv('csv/rooms.csv')

In [ ]:
arrays = {col: fmipa[col].to_numpy() for col in fmipa.columns}

course_id = arrays["course_id"]
course_code = arrays["course_code"]
duration = arrays["duration"]
time_slot = arrays["time_slot"]
room_id = arrays["room_id"]
room = arrays["room"]

In [ ]:
C = 51
T = 50
R = 54

# Generate solution
for i in range(10):
    export_to_txt(generate_valid_solution(fmipa, T, R), "solutions", f"solution_{i+1}.txt")

In [ ]:
# Check if solution is valid
for i in range(10):
    print(f"Solution {i+1}")
    arr = import_from_txt("solutions", f"solution_{i+1}.txt")

    criteria_list = [
        time_slot_criteria,
    ]

    is_valid = is_solution_valid(arr, criteria_list, course_df=fmipa)
    print("Is solution valid?", is_valid)

    if (is_valid):
        solution_dict = array_to_multi_position_dict(arr)
        used_rooms = get_used_rooms(solution_dict)
        print("Used rooms:", len(used_rooms))
    print()

In [ ]:
arr = import_from_txt("solutions", "solution_1.txt")
solution_dict = array_to_multi_position_dict(arr)
solution_df = dict_to_course_df(solution_dict, course_code)

solution_df = solution_df.merge(time_slots, left_on='time_slot_id', right_on='id', how='left')
# solution_df = solution_df.drop(columns=['id'])

solution_df = solution_df.merge(rooms[['id', 'room_name']], left_on='room_id', right_on='id', how='left')
solution_df = solution_df.drop(columns=['id_y'])

solution_df = solution_df.drop(columns=['time_slot_id', 'room_id'])

solution_df = add_time_range_column(solution_df)
solution_df.rename(columns={'id_x': 'time_slot_id'}, inplace=True)

In [ ]:
combinations = get_combinations([6, 6, 6, 5, 5])

# Order of courses matches combination order
course_order = ["MA1201", "FI1201", "KI1201", "KU1202", "KU1024"]

In [ ]:
for combo in combinations:
    # Suppose your combination is:
    # combo = (2, 2, 2, 4, 4)

    print(combo)

    # Get the rows:
    filtered_df = filter_classes_by_combination(solution_df, course_order, combo)

    ordered_schedule = group_schedule_by_day(filtered_df)

    for key, value in ordered_schedule.items():
        print(f"{key}:")
        for entry in value:
            print(f"  {entry}")
    print()